# 4. Grouping data

Dalam bab ini, Anda akan belajar bagaimana mengidentifikasi dan membagi DataFrames berdasarkan kelompok atau kategori untuk agregasi atau analisis lebih lanjut. Anda juga akan belajar cara mengubah dan memfilter data Anda, dan bagaimana mendeteksi outliers dan menghubungkan nilai yang hilang. Mengetahui cara mengelompokkan data dalam pandas secara efektif dapat menjadi tambahan yang sangat kuat bagi alat ilmu data Anda.

## Categoricals and groupby

### Advantages of categorical data types

Apa keuntungan utama menyimpan data secara eksplisit sebagai tipe categorical daripada tipe object?

* Komputasi lebih cepat dan data categorical membutuhkan lebih sedikit ruang dalam memori.

### Grouping by multiple columns

Dalam latihan ini, Anda akan kembali bekerja dengan dataset Titanic dari Bab 1 dan menggunakan `.groupby()` untuk menganalisis distribusi penumpang yang naik Titanic.

Kolom `'pclass'` mengidentifikasi kelas tiket mana yang dibeli oleh penumpang dan kolom `'embarked'` menunjukkan di mana dari tiga pelabuhan penumpang naik Titanic. `'S'` adalah singkatan dari Southampton, Inggris, `'C'` untuk Cherbourg, Prancis dan `'Q'` untuk Queenstown, Irlandia.

Tugas Anda adalah mengelompokkan pertama berdasarkan kolom `'pclass'` dan menghitung jumlah baris di setiap kelas menggunakan kolom `'survived'`. Anda kemudian akan mengelompokan berdasarkan kolom `'embarked'` dan `'pclass'` dan menghitung jumlah penumpang.

In [2]:
# Import pandas
import pandas as pd

# File url
fileurl = 'https://assets.datacamp.com/production/repositories/502/datasets/e280ed94bf4539afb57d8b1cbcc14bcf660d3c63/titanic.csv'

# Load data
titanic = pd.read_csv(fileurl)

# Print data
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
# Group titanic by 'pclass'
by_class = titanic.groupby('pclass')

# Aggregate 'survived' column of by_class by count
count_by_class = by_class['survived'].count()

# Print count_by_class
print(count_by_class)

# Group titanic by 'embarked' and 'pclass'
by_mult = titanic.groupby(['embarked', 'pclass'])

# Aggregate 'survived' column of by_mult by count
count_mult = by_mult['survived'].count()

# Print count_mult
print(count_mult)

pclass
1    323
2    277
3    709
Name: survived, dtype: int64
embarked  pclass
C         1         141
          2          28
          3         101
Q         1           3
          2           7
          3         113
S         1         177
          2         242
          3         495
Name: survived, dtype: int64


**Note** : Mengelompokkan data Anda dengan kolom tertentu seperti ini dan menggabungkannya dengan kolom lain, dalam hal ini, `'survived'`, memungkinkan Anda memeriksa data dengan cermat untuk mendapatkan wawasan yang menarik.

### Grouping by another series

Dalam latihan ini, Anda akan menggunakan dua set data dari Gapminder.org untuk menyelidiki harapan hidup rata-rata (dalam tahun) saat lahir pada 2010 untuk 6 wilayah benua. Untuk melakukan ini, Anda akan membaca data harapan hidup per negara menjadi satu pandas DataFrame dan hubungan antara negara dan wilayah ke negara lain.

Dengan mengatur indeks kedua DataFrames ke nama negara, Anda kemudian akan menggunakan informasi wilayah untuk mengelompokkan negara-negara dalam DataFrame harapan hidup dan menghitung nilai rata-rata untuk 2010.

File CSV harapan hidup tersedia untuk Anda di variabel `life_fname` dan nama file region tersedia di variabel `region_fname`.

In [5]:
# File url
life_fname = 'https://s3.amazonaws.com/assets.datacamp.com/production/course_1650/datasets/life_expectancy.csv'
regions_fname = 'https://s3.amazonaws.com/assets.datacamp.com/production/course_1650/datasets/regions.csv'

In [6]:
# Read life_fname into a DataFrame: life
life = pd.read_csv(life_fname, index_col='Country')

# Read regions_fname into a DataFrame: regions
regions = pd.read_csv(regions_fname, index_col='Country')

# Group life by regions['region']: life_by_region
life_by_region = life.groupby(regions['region'])

# Print the mean over the '2010' column of life_by_region
print(life_by_region['2010'].mean())

region
America                       74.037350
East Asia & Pacific           73.405750
Europe & Central Asia         75.656387
Middle East & North Africa    72.805333
South Asia                    68.189750
Sub-Saharan Africa            57.575080
Name: 2010, dtype: float64


**Note** : Sepertinya harapan hidup rata-rata (dalam tahun) saat lahir pada tahun 2010 adalah tertinggi di **Eropa & Asia Tengah** dan terendah di **Afrika Sub-Sahara**.

## Groupby and aggregation

### Computing multiple aggregates of multiple columns

Metode `.agg()` dapat digunakan dengan tuple atau list agregasi sebagai input. Saat menerapkan beberapa agregasi pada beberapa kolom, DataFrame teragregasi memiliki *indeks kolom multi-level*.

Dalam latihan ini, Anda akan mengelompokkan penumpang di Titanic dengan `'pclass'` dan mengagregasi kolom `'age'` dan `'fare'` dengan fungsi `'max'` dan `'median'`. Anda kemudian akan menggunakan pilihan multi-level untuk menemukan penumpang tertua per kelas dan harga rata-rata tarif per kelas.

In [7]:
# Group titanic by 'pclass': by_class
by_class = titanic.groupby('pclass')

# Select 'age' and 'fare'
by_class_sub = by_class[['age','fare']]

# Aggregate by_class_sub by 'max' and 'median': aggregated
aggregated = by_class_sub.agg(['max', 'median'])

# Print the maximum age in each class
print(aggregated.loc[:, ('age','max')])

# Print the median fare in each class
print(aggregated.loc[:, ('fare', 'median')])

pclass
1    80.0
2    70.0
3    74.0
Name: (age, max), dtype: float64
pclass
1    60.0000
2    15.0458
3     8.0500
Name: (fare, median), dtype: float64


**Note** : Tidak mengherankan bahwa tarif rata-rata tertinggi adalah untuk kelas penumpang pertama.

### Aggregating on index levels/fields

Jika Anda memiliki DataFrame dengan indeks baris multi-level, level individu dapat digunakan untuk melakukan groupby. Ini memungkinkan teknik agregasi lanjutan untuk diterapkan di sepanjang satu atau lebih level dalam indeks dan di satu atau lebih kolom.

Dalam latihan ini Anda akan menggunakan dataset Gapminder lengkap yang berisi nilai-nilai harapan hidup, populasi, angka kematian anak (per 1.000) tahunan dan produk domestik bruto (PDB) per kapita untuk setiap negara di dunia dari tahun 1964 hingga 2013.

Tugas Anda adalah membuat DataFrame multi-level pada kolom `'Tahun'`, `'Region'` dan `'Country'`. Selanjutnya Anda akan mengelompokkan DataFrame berdasarkan level `'Year'` dan `'Region'`. Terakhir, Anda akan menerapkan agregasi dictionary untuk menghitung total populasi, penyebaran nilai PDB per kapita, dan rata-rata angka kematian anak.

In [10]:
# File url
fileurl = 'https://assets.datacamp.com/production/repositories/502/datasets/09378cc53faec573bcb802dce03b01318108a880/gapminder_tidy.csv'

In [19]:
# Read the CSV file into a DataFrame and sort the index: gapminder
gapminder = pd.read_csv(fileurl, index_col=['Year','region','Country']).sort_index()
gapminder.head()

fertility    life  population  \
Year region  Country                                              
1964 America Antigua and Barbuda      4.250  63.775     58653.0   
             Argentina                3.068  65.388  21966478.0   
             Aruba                    4.059  67.113     57031.0   
             Bahamas                  4.220  64.189    133709.0   
             Barbados                 4.094  62.819    234455.0   

                                  child_mortality      gdp  
Year region  Country                                        
1964 America Antigua and Barbuda            72.78   5008.0  
             Argentina                      57.43   8227.0  
             Aruba                            NaN   5505.0  
             Bahamas                        48.56  18160.0  
             Barbados                       64.70   5681.0

In [20]:
# Group gapminder by 'Year' and 'region': by_year_region
by_year_region = gapminder.groupby(level=['Year', 'region'])

# Define the function to compute spread: spread
def spread(series):
    return series.max() - series.min()

# Create the dictionary: aggregator
aggregator = {'population':'sum', 'child_mortality':'mean', 'gdp':spread}

# Aggregate by_year_region using the dictionary: aggregated
aggregated = by_year_region.agg(aggregator)

# Print the last 6 entries of aggregated 
print(aggregated.tail(6))

                                   population  child_mortality       gdp
Year region                                                             
2013 America                     9.629087e+08        17.745833   49634.0
     East Asia & Pacific         2.244209e+09        22.285714  134744.0
     Europe & Central Asia       8.968788e+08         9.831875   86418.0
     Middle East & North Africa  4.030504e+08        20.221500  128676.0
     South Asia                  1.701241e+09        46.287500   11469.0
     Sub-Saharan Africa          9.205996e+08        76.944490   32035.0


**Note** : Apakah Anda dapat melihat korelasi antara `population`, `child_mortality`, dan `gdp`?

### Grouping on a function of the index

Operasi Groupby juga dapat dilakukan pada transformasi nilai indeks. Dalam kasus DateTimeIndex, kita bisa mengekstrak bagian dari datetime yang akan dikelompokkan.

Dalam latihan ini Anda akan membaca satu set sampel data penjualan mulai Februari 2015 dan menetapkan kolom `'Date'` sebagai indeks. Pekerjaan Anda adalah mengelompokkan data penjualan berdasarkan hari dalam seminggu dan mengagregasi jumlah kolom `'Units'`.

Apakah ada hari dalam seminggu yang lebih populer bagi pelanggan? Untuk mengetahuinya, Anda akan menggunakan `.strftime('%a')` untuk mengubah nilai indeks datetime menjadi hari yang disingkat dalam seminggu.

In [22]:
# Read file: sales
sales = pd.read_csv('sales-feb-2015.csv', parse_dates=True, index_col='Date')

# Create a groupby object: by_day
by_day = sales.groupby(sales.index.strftime('%a'))

# Create sum: units_sum
units_sum = by_day['Units'].sum()

# Print units_sum
print(units_sum)

Mon    48
Sat     7
Thu    59
Tue    13
Wed    48
Name: Units, dtype: int64


**Note** : Sepertinya hari Senin, Rabu, dan Kamis adalah hari yang paling populer bagi pelanggan!

## Groupby and transformation

### Detecting outliers with Z-Scores

Seperti Dhavide ditunjukkan dalam video menggunakan fungsi `zscore`, Anda dapat menerapkan metode `.transform()` setelah pengelompokan untuk menerapkan fungsi ke grup data secara independen. Z-score juga berguna untuk menemukan outlier: nilai z-score dari +/- 3 umumnya dianggap sebagai outlier.

Dalam contoh ini, Anda akan menormalisasi data Gapminder pada 2010 untuk harapan hidup dan kesuburan dengan z-score per wilayah. Dengan menggunakan pengindeksan boolean, Anda akan memfilter untuk negara-negara yang memiliki tingkat kesuburan tinggi dan harapan hidup yang rendah untuk wilayah mereka.

Gapminder DataFrame tahun 2010 yang diindeks oleh `'Country'` disediakan untuk Anda sebagai `gapminder_2010`.

In [39]:
# Assign file url
fileurl = 'https://assets.datacamp.com/production/repositories/502/datasets/09378cc53faec573bcb802dce03b01318108a880/gapminder_tidy.csv'

# Load data
gapminder_2010 = pd.read_csv(fileurl, index_col='Country')
gapminder_2010.head()

,Year,fertility,life,population,child_mortality,gdp,region
Country,,,,,,,
Afghanistan,1964,7.671,33.639,10474903.0,339.7,1182.0,South Asia
Afghanistan,1965,7.671,34.152,10697983.0,334.1,1182.0,South Asia
Afghanistan,1966,7.671,34.662,10927724.0,328.7,1168.0,South Asia
Afghanistan,1967,7.671,35.170,11163656.0,323.3,1173.0,South Asia
Afghanistan,1968,7.671,35.674,11411022.0,318.1,1187.0,South Asia


In [26]:
# Import zscore
from scipy.stats import zscore

# Group gapminder_2010: standardized
standardized = gapminder_2010.groupby('region')['life', 'fertility'].transform(zscore)

# Construct a Boolean Series to identify outliers: outliers
outliers = (standardized['life'] < -3) | (standardized['fertility'] > 3)

# Filter gapminder_2010 by the outliers: gm_outliers
gm_outliers = gapminder_2010.loc[outliers]

# Print gm_outliers
gm_outliers

,Year,fertility,life,population,child_mortality,gdp,region
Country,,,,,,,
Bolivia,1964,6.607,43.913,3668568.0,265.40,2971.0,America
Bolivia,1965,6.593,44.229,3752892.0,260.10,3046.0,America
Bolivia,1966,6.586,44.536,3839751.0,254.50,3191.0,America
Bolivia,1967,6.585,44.835,3929192.0,248.80,3312.0,America
Bolivia,1968,6.587,45.132,4021551.0,243.00,3510.0,America
...,...,...,...,...,...,...,...
Turkey,1976,4.872,56.050,40446729.0,152.70,9142.0,Europe & Central Asia
Turkmenistan,1964,6.663,56.181,1829697.0,132.98,7952.0,Europe & Central Asia
"Yemen, Rep.",1964,7.385,36.068,5527652.0,NaN,NaN,Middle East & North Africa


**Note** : Menggunakan z-score seperti ini adalah cara terbaik untuk mengidentifikasi outlier dalam data Anda.

### Filling missing data (imputation) by group

Banyak paket statistik dan pembelajaran mesin tidak dapat menentukan tindakan terbaik yang harus diambil ketika entri data yang hilang ditemukan. Berurusan dengan data yang hilang adalah alami dalam pandas (baik dalam menggunakan perilaku default dan dalam mendefinisikan perilaku kustom). Di Bab 1, Anda berlatih menggunakan metode `.dropna()` untuk menjatuhkan nilai yang hilang. Sekarang, Anda akan berlatih memasukkan nilai-nilai yang hilang. Anda dapat menggunakan `.groupby()` dan `.transform()` untuk mengisi data yang hilang dengan tepat untuk setiap grup.

Tugas Anda adalah mengisi nilai `'age'` yang hilang untuk penumpang di Titanic dengan usia rata-rata dari `'gender'` dan `'pclass'`. Untuk melakukan ini, Anda akan mengelompokkan menurut kolom `'sex'` dan `'pclass'` dan mengubah setiap kelompok dengan fungsi kustom untuk memanggil `.fillna()` dan memasukkan nilai median.

DataFrame telah dimuat sebelumnya sebagai `titanic`. Jelajahi di IPython Shell dengan mencetak output dari `titanic.tail(10)`. Perhatikan khususnya `NaN` di kolom `'age'`.

In [27]:
# Assign file url
fileurl = 'https://assets.datacamp.com/production/repositories/502/datasets/e280ed94bf4539afb57d8b1cbcc14bcf660d3c63/titanic.csv'

# Load data
titanic = pd.read_csv(fileurl)
titanic.tail(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1299,3,0,"Yasbeck, Mr. Antoni",male,27.0,1,0,2659,14.4542,NaN,C,C,NaN,NaN
1300,3,1,"Yasbeck, Mrs. Antoni (Selini Alexander)",female,15.0,1,0,2659,14.4542,NaN,C,NaN,NaN,NaN
1301,3,0,"Youseff, Mr. Gerious",male,45.5,0,0,2628,7.2250,NaN,C,NaN,312.0,NaN
1302,3,0,"Yousif, Mr. Wazli",male,NaN,0,0,2647,7.2250,NaN,C,NaN,NaN,NaN
1303,3,0,"Yousseff, Mr. Gerious",male,NaN,0,0,2627,14.4583,NaN,C,NaN,NaN,NaN
1304,3,0,"Zabour, Miss. Hileni",female,14.5,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN
1308,3,0,"Zimmerman, Mr. Leo",male,29.0,0,0,315082,7.8750,NaN,S,NaN,NaN,NaN


In [29]:
# Create a groupby object: by_sex_class
by_sex_class = titanic.groupby(['sex', 'pclass'])

# Write a function that imputes median
def impute_median(series):
    return series.fillna(series.median())

# Impute age and assign to titanic['age']
titanic.age = by_sex_class.age.transform(impute_median)

# Print the output of titanic.tail(10)
titanic.tail(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1299,3,0,"Yasbeck, Mr. Antoni",male,27.0,1,0,2659,14.4542,NaN,C,C,NaN,NaN
1300,3,1,"Yasbeck, Mrs. Antoni (Selini Alexander)",female,15.0,1,0,2659,14.4542,NaN,C,NaN,NaN,NaN
1301,3,0,"Youseff, Mr. Gerious",male,45.5,0,0,2628,7.2250,NaN,C,NaN,312.0,NaN
1302,3,0,"Yousif, Mr. Wazli",male,25.0,0,0,2647,7.2250,NaN,C,NaN,NaN,NaN
1303,3,0,"Yousseff, Mr. Gerious",male,25.0,0,0,2627,14.4583,NaN,C,NaN,NaN,NaN
1304,3,0,"Zabour, Miss. Hileni",female,14.5,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,22.0,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN
1308,3,0,"Zimmerman, Mr. Leo",male,29.0,0,0,315082,7.8750,NaN,S,NaN,NaN,NaN


**Note** : Menempatkan nilai-nilai yang hilang secara cerdas selalu lebih baik daripada menjatuhkannya sepenuhnya!

### Other transformations with .apply

Metode `.apply()` ketika digunakan pada objek groupby melakukan fungsi *arbitrary* pada masing-masing grup. Fungsi-fungsi ini dapat berupa agregasi, transformasi atau alur kerja yang lebih kompleks. Metode `.apply()` kemudian akan menggabungkan hasilnya dengan cara yang cerdas.

Dalam latihan ini, Anda akan menganalisis kesenjangan ekonomi di dalam kawasan di dunia menggunakan dataset Gapminder untuk tahun 2010. Untuk melakukan ini, Anda akan mendefinisikan fungsi untuk menghitung penyebaran agregat PDB per kapita di setiap wilayah dan negara masing-masing z-score PDB per kapita regional. Anda kemudian akan memilih tiga negara - Amerika Serikat, Inggris Raya dan Cina - untuk melihat ringkasan PDB regional dan z-score negara tersebut terhadap rata-rata regional.

Fungsi berikut telah ditentukan untuk penggunaan Anda:

In [55]:
# Assign file url
fileurl = 'https://assets.datacamp.com/production/repositories/502/datasets/09378cc53faec573bcb802dce03b01318108a880/gapminder_tidy.csv'

# Load data
gapminder_2010 = pd.read_csv(fileurl, index_col='Country').sort_index()
# Deleted duplicated index
gapminder_2010 = gapminder_2010[~gapminder_2010.index.duplicated()]

In [56]:
def disparity(gr):
    # Compute the spread of gr['gdp']: s
    s = gr['gdp'].max() - gr['gdp'].min()
    # Compute the z-score of gr['gdp'] as (gr['gdp']-gr['gdp'].mean())/gr['gdp'].std(): z
    z = (gr['gdp'] - gr['gdp'].mean())/gr['gdp'].std()
    # Return a DataFrame with the inputs {'z(gdp)':z, 'regional spread(gdp)':s}
    return pd.DataFrame({'z(gdp)':z , 'regional spread(gdp)':s})

In [57]:
# Group gapminder_2010 by 'region': regional
regional = gapminder_2010.groupby('region')

# Apply the disparity function on regional: reg_disp
reg_disp = regional.apply(disparity)

# Print the disparity of 'United States', 'United Kingdom', and 'China'
print(reg_disp.loc[['United States', 'United Kingdom', 'China']])

                  z(gdp)  regional spread(gdp)
Country                                       
United States   2.935012               18314.0
United Kingdom  0.896839               28734.0
China          -0.432686               66821.0


## Groupby and filtering

### Grouping and filtering with .apply()

Dengan menggunakan `.apply()`, Anda dapat menulis fungsi yang memfilter baris dalam grup. Metode `.apply()` akan menangani iterasi di atas grup individu dan kemudian menggabungkannya kembali menjadi Series atau DataFrame.

Dalam latihan ini Anda akan mengambil kumpulan data Titanic dan menganalisis tingkat kelangsungan hidup dari dek `'C'`, yang berisi penumpang terbanyak. Untuk melakukan ini, Anda akan mengelompokkan dataset berdasarkan `'sex'` dan kemudian menggunakan metode `.apply()` pada fungsi yang ditentukan pengguna yang menghitung tingkat kelangsungan hidup rata-rata pada dek `'C'`:

In [61]:
def c_deck_survival(gr):

    c_passengers = gr['cabin'].str.startswith('C').fillna(False)

    return gr.loc[c_passengers, 'survived'].mean()

In [62]:
# Assign file url
fileurl = 'https://assets.datacamp.com/production/repositories/502/datasets/e280ed94bf4539afb57d8b1cbcc14bcf660d3c63/titanic.csv'

# Load data
titanic = pd.read_csv(fileurl)

In [63]:
# Create a groupby object using titanic over the 'sex' column: by_sex
by_sex = titanic.groupby('sex')

# Call by_sex.apply with the function c_deck_survival
c_surv_by_sex = by_sex.apply(c_deck_survival)

# Print the survival rates
print(c_surv_by_sex)

sex
female    0.913043
male      0.312500
dtype: float64


**Note** : Sepertinya penumpang wanita di dek `'C'` memiliki peluang lebih tinggi untuk bertahan hidup!

### Grouping and filtering with .filter()

Anda bisa menggunakan groupby dengan metode `.filter()` untuk menghapus seluruh grup baris dari DataFrame berdasarkan kondisi boolean.

Dalam latihan ini, Anda akan mengambil data penjualan Februari dan menghapus entri dari perusahaan yang membeli kurang dari atau sama dengan 35 Unit dalam sebulan penuh.

Pertama, Anda akan mengidentifikasi berapa unit yang dibeli masing-masing perusahaan untuk verifikasi. Selanjutnya Anda akan menggunakan metode `.filter()` setelah pengelompokan oleh `'Company'` untuk menghapus semua baris milik perusahaan yang jumlahnya di kolom `'Units'` kurang dari atau sama dengan 35. Akhirnya, verifikasi bahwa tiga perusahaan dengan total Unit dibeli kurang dari atau sama dengan 35 telah disaring dari DataFrame.

In [64]:
# Read the CSV file into a DataFrame: sales
sales = pd.read_csv('sales-feb-2015.csv', index_col='Date', parse_dates=True)

# Group sales by 'Company': by_company
by_company = sales.groupby('Company')

# Compute the sum of the 'Units' of by_company: by_com_sum
by_com_sum = by_company['Units'].sum()
print(by_com_sum)

# Filter 'Units' where the sum is > 35: by_com_filt
by_com_filt = by_company.filter(lambda g:g['Units'].sum() > 35)
print(by_com_filt)

Company
Acme Coporation    34
Hooli              30
Initech            30
Mediacore          45
Streeplex          36
Name: Units, dtype: int64
                       Company   Product  Units
Date                                           
2015-02-02 21:00:00  Mediacore  Hardware      9
2015-02-04 15:30:00  Streeplex  Software     13
2015-02-09 09:00:00  Streeplex   Service     19
2015-02-09 13:00:00  Mediacore  Software      7
2015-02-19 11:00:00  Mediacore  Hardware     16
2015-02-19 16:00:00  Mediacore   Service     10
2015-02-21 05:00:00  Mediacore  Software      3
2015-02-26 09:00:00  Streeplex   Service      4


### Filtering and grouping with .map()

Anda telah melihat cara mengelompokkan berdasarkan satu kolom, atau beberapa kolom. Terkadang, Anda mungkin ingin mengelompokkan berdasarkan fungsi / transformasi kolom. Kuncinya di sini adalah bahwa Series diindeks dengan cara yang sama seperti DataFrame. Anda juga dapat mencampur dan mencocokkan pengelompokan kolom dengan pengelompokan Series.

Dalam latihan ini, tugas Anda adalah menyelidiki tingkat kelangsungan hidup penumpang di Titanic berdasarkan `'age'` dan `'pclass'`. Secara khusus, tujuannya adalah untuk mengetahui fraksi apa yang anak-anak di bawah 10 bertahan di setiap `'pclass'`. Anda akan melakukan ini dengan terlebih dahulu membuat array boolean di mana `True` adalah penumpang di bawah 10 tahun dan `False` adalah penumpang di atas 10. Anda akan menggunakan `.map()` untuk mengubah nilai-nilai ini menjadi string.

Akhirnya, Anda akan mengelompokkan menurut series di bawah 10 dan kolom `'pclass'` dan mengagregasi kolom `'survived'`. Kolom `'survived'` memiliki nilai `1` jika penumpang selamat dan `0` sebaliknya. Rata-rata dari kolom `'survived'` adalah sebagian kecil penumpang yang hidup.

In [65]:
# Create the Boolean Series: under10
under10 = (titanic['age'] < 10).map({True:'under 10', False:'over 10'})

# Group by under10 and compute the survival rate
survived_mean_1 = titanic.groupby(under10)['survived'].mean()
print(survived_mean_1)

# Group by under10 and pclass and compute the survival rate
survived_mean_2 = titanic.groupby([under10, 'pclass'])['survived'].mean()
print(survived_mean_2)

age
over 10     0.366748
under 10    0.609756
Name: survived, dtype: float64
age       pclass
over 10   1         0.617555
          2         0.380392
          3         0.238897
under 10  1         0.750000
          2         1.000000
          3         0.446429
Name: survived, dtype: float64


**Note** : Sepertinya penumpang di bawah usia 10 memiliki tingkat kelangsungan hidup yang lebih tinggi daripada mereka yang berusia di atas 10.